In [ ]:
import sys
sys.path.append('../')

In [ ]:
from deepracer.tracks import TrackIO, Track

from deepracer.logs import \
    AnalysisUtils as au, \
    SimulationLogsIO as slio, \
    EvaluationUtils as eu, \
    PlottingUtils as pu,\
    NewRewardUtils as nr \

In [ ]:
tu = TrackIO(base_path='../tracks')
track: Track = tu.load_track("reinvent_base")

track.road_poly

In [ ]:
fname = '../logs/evaluation-simulation-logs-w6.log'
df = slio.load_pandas(fname)
df = df.rename(columns={'track_len':'track_length'}) 
df['track_width'] = 0.76
df.head()

In [ ]:
nr.new_reward(df, track.center_line, 'reward') #, verbose=True)

In [ ]:
df.plot(x='reward',y='new_reward',kind='scatter')

In [ ]:
from reward import AWSReward, RewardFactory
factory = RewardFactory(AWSReward)
factory.print_available_reward_classes()

In [ ]:
params = nr.df_to_params(next(df.iterrows())[1], track.center_line)
missing_params = {'track_width': 0.76, 'track_length': 17.67}
params.update(missing_params)
w6 = factory.build('RewardW6',params)
float(w6)

In [ ]:
# This util has a very complicated way to get a reward function
nr.new_reward??

In [ ]:
#this util is missing some parameters and hardcoding others
nr.df_to_params??

In [ ]:
def new_reward(df, factory, center_line, reward_name='RewardX1',missing_params=None):
        new_rewards = []
        for _, row in df.iterrows():
            params = nr.df_to_params(row, center_line)
            if missing_params:
                params.update(missing_params)
            rw = factory.build(reward_name,params)
            new_rewards.append( float(rw) )

        df[reward_name] = new_rewards

In [ ]:
new_reward(df,factory,track.center_line,reward_name='RewardW6',missing_params=missing_params)

In [ ]:
df.head()